In [1]:
import pandas as pd 
import numpy as np

In [ ]:
news_df = pd.read_csv("./dataset/news-coindesk-backup.csv")
stock_df = pd.read_csv("./dataset/btcprice2019-2024.csv")

for i in range(len(stock_df)):
    date = stock_df['Date'][i][:10]
    stock_df['Date'][i] = date

In [7]:
news_df = news_df[news_df['Date'].isin(stock_df['Date'].tolist())]

news_df.to_csv("./dataset/news_data.csv", index=False)

Ensured all the dates between data and news are the same


Now, I want to collate all news to a date

In [15]:
# Group by the first column (date) and concatenate titles and descriptions
collapsed_df = news_df.groupby("Date").agg(lambda x: ', '.join(x.astype(str))).reset_index()

In [30]:
# Combine Title and Description, removing any leading/trailing whitespace
news_df['News'] = news_df['Title'] + ' ' + news_df['Description'].fillna('')
news_df['News'] = news_df['News'].str.strip()

# Group by Date and aggregate News
grouped = news_df.groupby('Date')['News'].apply(list).reset_index()


In [31]:
grouped

,Date,News
0,2019-01-01,[0 0]
1,2019-01-02,"[0 0, Down More than 70% in 2018, Bitcoin Clos..."
2,2019-01-03,"[0 0, Overstock Will Pay Some of Its 2019 Taxe..."
3,2019-01-04,"[0 0, $4K Ahead? Bitcoin's Low-Volume Price Pu..."
4,2019-01-05,"[0 0, Bitcoin Payments Aren't Dead, They've Ju..."
...,...,...
2071,2024-09-26,"[0 0, Crypto for Advisors: The Growth of Bitco..."
2072,2024-09-27,"[0 0, The Paradox of the Bitcoin Maximalist Wh..."
2073,2024-09-28,[0 0]
2074,2024-09-29,[0 0]


In [32]:
# Create new columns for each news item
max_news = grouped['News'].apply(len).max()
for i in range(1, max_news + 1):
    grouped[f'News {i}'] = grouped['News'].apply(lambda x: x[i-1] if i <= len(x) else '')

# Drop the original 'News' column
grouped = grouped.drop('News', axis=1)

# Rename columns to match desired output
grouped.columns = ['Date'] + [f'News {i}' for i in range(1, max_news + 1)]

# Remove any columns that are all empty (all zeros)
grouped = grouped.loc[:, (grouped != '').any()]

# Reset the index
grouped = grouped.reset_index(drop=True)


In [42]:
grouped = grouped.drop(columns=['News 1'])

In [46]:
len(grouped)

2076

In [47]:
grouped = grouped.replace(r'^\s*$', 0, regex=True)

In [50]:
grouped.to_csv("./dataset/news_data_cleaned.csv", index=False)